In this notebook, we develop a pipeline for turning fMRI data into pseudo-fNIRS data, and we apply it to the NSD.

In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

In [2]:
# specify subject data
subjectID  = '01'
sessionID  = '21'
runID      = '01'

# create directories to store data
anat_path = 'data/sub' + subjectID + '/anat/'
run_path  = 'data/sub' + subjectID + '/fmri/sess' + sessionID + '/run' + runID + '/'
if not os.path.isdir(anat_path): os.makedirs(anat_path)
if not os.path.isdir(run_path):  os.makedirs(run_path)

# 1. Create segmented mask

### Install FSL library

In [22]:
## uncomment if library not installed
# !sudo apt -qq install file
# !wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
# !python3 fslinstaller.py

from pathlib import Path
fslpath = Path(os.getcwd()).parents[1].as_posix()+'/fsl'
os.environ["FSLDIR"] = fslpath
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

### Retrieve subject data

In [32]:
resolution = '1pt0'

# specify filenames
anat_url = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata/ppdata/subj{subjectID}/anat/"
head_mask_filename = f"brainmask_{resolution}.nii.gz"
t1_filename = f"T1_{resolution}_masked.nii.gz"

# pull structural data from NSD
if not os.path.isfile(anat_path+head_mask_filename): 
    !wget -P $anat_path $anat_url$head_mask_filename 
if not os.path.isfile(anat_path+t1_filename): 
    !wget -P $anat_path $anat_url$t1_filename

# generate brain masks (~5s)
t1_brain_filename = f"T1_{resolution}_masked_brain"
if not os.path.isfile(anat_path+t1_brain_filename+'.nii.gz'):
    !bet $anat_path$t1_filename $anat_path$t1_brain_filename -m

# segment brain tissue into csf, grey and white matter (~5m8s)
if not os.path.isfile(anat_path+t1_brain_filename+'_seg.nii.gz'):
    !fast $anat_path$t1_brain_filename

In [ ]:
# specify filenames
fmri_path = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata_rawdata/sub-{subjectID}/ses-nsd{sessionID}/func/"
fmri_file = f"sub-{subjectID}_ses-nsd{sessionID}_task-nsdcore_run-{runID}_bold.nii.gz"
resting_state_path = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata_rawdata/sub-{subjectID}/ses-nsd{sessionID}/func/"
resting_state_file = f"sub-{subjectID}_ses-nsd{sessionID}_task-rest_run-{1}_bold.nii.gz"

# pull functional data from NSD
!wget $fmri_path$fmri_file $resting_state_path$resting_state_file